## Capstone 2 Project: Prediction of WAR decline in MLB players

In [1]:
import csv
import pandas as pd
import numpy as np
import fnmatch
import re
import warnings
import operator
from collections import defaultdict

# read Baseball Prospectus and Lahman csv files and convert to dataframes
BB_Prospectus_csvfile = 'bpstats_02-02-2019_year.csv'
Lahman_Fielding_csvfile = 'Lahman_Fielding.csv'

bb_prosp = pd.read_csv(BB_Prospectus_csvfile)
lahman_fielding = pd.read_csv(Lahman_Fielding_csvfile)


# updates to Baseball Prospectus dataframe
bb_prosp = bb_prosp[(bb_prosp['YEAR'] >= 1977) & (bb_prosp['YEAR'] <= 2017) & (bb_prosp['AB'] > 50)]   # include seasons 1977-2017
bb_prosp = bb_prosp.drop('HBP', axis=1)   # drop HBP column
bb_prosp['playerID'] = np.nan   # create playerID column for player name conversion
bb_prosp.NAME = bb_prosp.NAME.str.replace('  ', ' ')  # fixes player names with double space b/w first and last name (ex. 'Alex  Bregman')
bb_prosp.index = range(0, len(bb_prosp))   # create indexing for dataframe
bb_prosp.rename(columns={'YEAR':'yearID', '#':'SampleID', 'DRAA':'Def_DRAA', 'FRAA':'Def_FRAA'}, inplace=True)   # renaming YEAR column to merge with Lahman dataframe


# updates to Lahman dataframe 
lahman_fielding['lahmanID'] = lahman_fielding['playerID']   
lahman_fielding['playerID'] = lahman_fielding['playerID'].str[:-2]   # drop the last two characters in playerID column
lahman_fielding.drop(columns=['G', 'stint'], axis=1, inplace=True)   # drop G and stint columns
lahman_fielding.rename(columns={'lgID':'League'}, inplace=True)   # renaming lgID column to League

def_stat_old = ['PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR']    # identifying defensive stats with prefix 'Def_'
for cols in def_stat_old:
    lahman_fielding.rename(columns={cols:'Def_'+cols}, inplace=True)

    
# disable warnings
warnings.filterwarnings('ignore')   


# stat columns
general_stats = ['SampleID', 'NAME', 'yearID', 'playerID', 'lahmanID', 'teamID', 'AGE', 'POS', 'G', 'GS', 'League']
offensive_stats = ['PA', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR', 'TB', 'BB', 'IBB', 'SO', 'SF', 'SH', 'RBI', 'DP', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'ISO', 'oppOPS', 'DRC+', 'DRC+ SD', 'BRR', 'BWARP']
agg_defensive_stats = ['InnOuts', 'Def_PO', 'Def_A', 'Def_E', 'Def_DP', 'Def_PB', 'Def_WP', 'Def_SB', 'Def_CS', 'Def_ZR']
non_agg_defensive_stats = ['Def_DRAA', 'Def_FRAA']

In [2]:
# function that takes Baseball Prospectus first name and last name and converts to Lahman naming convention

def convert_to_lahman_name(first_name, last_name):
    return last_name[0:5] + first_name[0:2]

In [3]:
%%time

# for loop that iterates through the NAME column in bb_prosp dataframe and passes first name and last name
# to convert_to_lahman_name function; result is stored in playerID column

for n in bb_prosp.index:
    
    if bb_prosp['NAME'].iloc[n].count(' ') == 3:   # checks and converts names with 3 whitespaces (ex. 'Tomas de la Rosa')
        name = bb_prosp['NAME'].iloc[n].partition(' ')
        first_name = name[0].lower()
        last_name = name[2].replace(' ', '').lower()
        
        convert_to_lahman_name(first_name, last_name)  
               
    elif bb_prosp['NAME'].iloc[n].count(' ') == 2:   # checks for names with 2 whitespaces
        
        if re.search(' Jr.', bb_prosp['NAME'].iloc[n]):   # converts names that end with Jr. (ex. 'Jose Cruz Jr.')
            name = bb_prosp['NAME'].iloc[n].split(' ')
            first_name = name[0].lower()
            last_name = name[1].replace(' ', '').lower()
            
            convert_to_lahman_name(first_name, last_name)
            
        elif re.search('\w\. \w\.', bb_prosp['NAME'].iloc[n]):   # converts names with initialed first name with whitespace (ex. 'J. T. Bruett')
            name = bb_prosp['NAME'].iloc[n].split(' ')
            first_name = (name[0].strip('.') + name[1].strip('.')).lower()
            last_name = name[2].lower()
            
            convert_to_lahman_name(first_name, last_name)
            
        elif re.search('\s\w\.\s', bb_prosp['NAME'].iloc[n]):   # converts names with middle initial (ex. 'Bobby J. Jones')
            name = bb_prosp['NAME'].iloc[n].split(' ')
            first_name = name[0].lower()
            last_name = name[2].lower()
            
            convert_to_lahman_name(first_name, last_name)
            
        else:   # converts names with multi-word last names (ex. 'Andy Van Slyke')
            name = bb_prosp['NAME'].iloc[n]
            
            exception_list_1 = ['Wily Mo Pena', 'Chan Ho Park', 'Jae Weong Seo', 'Billy Jo Robidoux']   # ignore middle name
            exception_list_2 = ['La Marr Hoyt']   # remove whitespace in first name
            exception_list_3 = ['John Ryan Murphy']   # goes by JR Murphy
            
            if name in exception_list_1:
                name = name.split(' ')
                first_name = name[0].lower()
                last_name = name[2].lower()
                
                convert_to_lahman_name(first_name, last_name)
                
            elif name in exception_list_2:
                name = name.split(' ')
                first_name = (name[0] + name[1]).lower()
                last_name = name[2].lower()
                
                convert_to_lahman_name(first_name, last_name)
                
            elif name in exception_list_3:
                name = name.split(' ')    
                first_name = (name[0][0] + name[1][0]).lower()
                last_name = name[2]
                
                convert_to_lahman_name(first_name, last_name)
                
            else:
                name = name.split(' ')
                first_name = name[0].lower()
                last_name = (name[1] + name[2]).lower()

                convert_to_lahman_name(first_name, last_name)
            
    elif bb_prosp['NAME'].iloc[n].count(' ') == 1:   # checks for names with 2 whitespaces
        
        if re.search('\.', bb_prosp['NAME'].iloc[n]):   # converts names with initialed first name with no whitespace (ex 'J.T. Snow')
            name = bb_prosp['NAME'].iloc[n].split(' ')
            first_name = name[0].replace('.','').lower()
            last_name = name[1].lower()
            
            convert_to_lahman_name(first_name, last_name)
            
        else:
            name = bb_prosp['NAME'].iloc[n].partition(' ')   # converts standard naming (ex. 'Barry Bonds')
            first_name = name[0].lower()
            last_name = name[2].lower()
            
    bb_prosp['playerID'][n] = convert_to_lahman_name(first_name, last_name)    

CPU times: user 9min 26s, sys: 2.93 s, total: 9min 29s
Wall time: 9min 54s


In [4]:
# merging updated Lahman and Baseball Prospectus dataframes

resultdf = pd.merge(bb_prosp, lahman_fielding, on=['playerID', 'yearID'], how='inner', indicator=True)
resultdf.columns

Index(['SampleID', 'NAME', 'yearID', 'AGE', 'G', 'PA', 'AB', 'R', 'H', '1B',
       '2B', '3B', 'HR', 'TB', 'BB', 'IBB', 'SO', 'SF', 'SH', 'RBI', 'DP',
       'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'ISO', 'oppOPS', 'DRC+',
       'DRC+ SD', 'Def_DRAA', 'BRR', 'Def_FRAA', 'BWARP', 'playerID', 'teamID',
       'League', 'POS', 'GS', 'InnOuts', 'Def_PO', 'Def_A', 'Def_E', 'Def_DP',
       'Def_PB', 'Def_WP', 'Def_SB', 'Def_CS', 'Def_ZR', 'lahmanID', '_merge'],
      dtype='object')

In [5]:
# sorting of resulting dataframe by player position, name, player season

resultdf.sort_values(['POS', 'playerID', 'yearID'], ascending=[True, True, True], inplace=True)
resultdf = resultdf[general_stats + offensive_stats + agg_defensive_stats + non_agg_defensive_stats]   # reorder columns
resultdf.head()

,SampleID,NAME,yearID,playerID,lahmanID,teamID,AGE,POS,G,GS,...,Def_A,Def_E,Def_DP,Def_PB,Def_WP,Def_SB,Def_CS,Def_ZR,Def_DRAA,Def_FRAA
29484,58864,Kurt Abbott,1999,abbotku,abbotku01,COL,30,1B,96,7.0,...,3,0.0,6,NaN,NaN,NaN,NaN,NaN,-8.4,-2.9
1218,1701,Jose Abreu,2014,abreujo,abreujo02,CHA,27,1B,145,109.0,...,69,6.0,105,NaN,NaN,NaN,NaN,NaN,39.4,-2.3
3218,4247,Jose Abreu,2015,abreujo,abreujo02,CHA,28,1B,154,115.0,...,60,11.0,100,NaN,NaN,NaN,NaN,NaN,19.5,6.2
3651,4730,Jose Abreu,2016,abreujo,abreujo02,CHA,29,1B,159,152.0,...,84,10.0,131,NaN,NaN,NaN,NaN,NaN,16.0,7.1
2248,3032,Jose Abreu,2017,abreujo,abreujo02,CHA,30,1B,156,138.0,...,78,8.0,130,NaN,NaN,NaN,NaN,NaN,23.5,5.5


In [6]:
# combining stats for players that have played for multiple teams in a season

stats_agg = dict()

for n in general_stats:
    stats_agg.update({n:'first'})

for n in offensive_stats:
    stats_agg.update({n:'first'})

for n in agg_defensive_stats:
    stats_agg.update({n:'sum'})
    
for n in non_agg_defensive_stats:
    stats_agg.update({n:'first'})
    
print(stats_agg)

# aggregating statistics for a player across multiple teams and groupby by player season and position
group_by_team_df = resultdf.groupby(['SampleID','POS']).agg(stats_agg)

{'SampleID': 'first', 'NAME': 'first', 'yearID': 'first', 'playerID': 'first', 'lahmanID': 'first', 'teamID': 'first', 'AGE': 'first', 'POS': 'first', 'G': 'first', 'GS': 'first', 'League': 'first', 'PA': 'first', 'AB': 'first', 'R': 'first', 'H': 'first', '1B': 'first', '2B': 'first', '3B': 'first', 'HR': 'first', 'TB': 'first', 'BB': 'first', 'IBB': 'first', 'SO': 'first', 'SF': 'first', 'SH': 'first', 'RBI': 'first', 'DP': 'first', 'SB': 'first', 'CS': 'first', 'AVG': 'first', 'OBP': 'first', 'SLG': 'first', 'OPS': 'first', 'ISO': 'first', 'oppOPS': 'first', 'DRC+': 'first', 'DRC+ SD': 'first', 'BRR': 'first', 'BWARP': 'first', 'InnOuts': 'sum', 'Def_PO': 'sum', 'Def_A': 'sum', 'Def_E': 'sum', 'Def_DP': 'sum', 'Def_PB': 'sum', 'Def_WP': 'sum', 'Def_SB': 'sum', 'Def_CS': 'sum', 'Def_ZR': 'sum', 'Def_DRAA': 'first', 'Def_FRAA': 'first'}


In [7]:
%%time

# for players that play more than one position in a season, assign position as the one he has played the most 
# inning outs.

records = []

for sample_id, temp_df in group_by_team_df.groupby(level=0):

    if len(temp_df) > 1:
        
        max_row = max(temp_df.iterrows(), key=lambda ind_row: ind_row[1].InnOuts)
        position_to_apply = max_row[1].POS
        
        new_records = [dict(row) for ind, row in temp_df.iterrows()]
        for new_record in new_records:
            new_record['POS'] = position_to_apply
            
        records.extend(new_records)
            
    else:
        
        ind, row = next(temp_df.iterrows())
        records.append(dict(row))
        
df = pd.DataFrame.from_records(records)

# aggregate statistics for a player that plays multiple positions in one season
final_df = df.groupby(['SampleID','POS']).agg(stats_agg)

CPU times: user 35.2 s, sys: 443 ms, total: 35.7 s
Wall time: 36.3 s


In [12]:
# removing pitchers and saving file to csv

final_df = final_df[final_df.POS != 'P']
final_df.to_csv('capstone_2_df.csv')

In [11]:
final_df.describe()

,SampleID,yearID,AGE,G,GS,PA,AB,R,H,1B,...,Def_A,Def_E,Def_DP,Def_PB,Def_WP,Def_SB,Def_CS,Def_ZR,Def_DRAA,Def_FRAA
count,18621.000000,18621.000000,18621.000000,18621.000000,18614.000000,18621.000000,18621.000000,18621.000000,18621.000000,18621.000000,...,18621.000000,18621.000000,18621.000000,18621.000000,18621.0,18621.000000,18621.000000,18621.0,18621.000000,18621.000000
mean,25185.360829,1997.790989,28.544761,99.282960,57.900720,355.724397,317.193813,42.797379,84.433596,57.371355,...,90.969604,6.101445,23.555341,0.702594,0.0,6.349981,2.862682,0.0,1.593808,0.122920
std,21249.428759,11.778648,4.120564,41.655386,52.898609,201.303607,178.736722,29.575826,53.056965,36.134389,...,126.369497,5.806656,33.315921,2.205342,0.0,18.042952,8.537772,0.0,11.987890,6.666083
min,3.000000,1977.000000,18.000000,14.000000,0.000000,51.000000,51.000000,0.000000,4.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,-39.600000,-58.000000
25%,8551.000000,1988.000000,25.000000,64.000000,9.000000,170.000000,152.000000,17.000000,36.000000,25.000000,...,6.000000,2.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,-4.600000,-2.900000
50%,17633.000000,1998.000000,28.000000,103.000000,40.000000,339.000000,303.000000,37.000000,77.000000,53.000000,...,30.000000,4.000000,7.000000,0.000000,0.0,0.000000,0.000000,0.0,-1.500000,-0.200000
75%,38000.000000,2008.000000,31.000000,137.000000,105.000000,535.000000,477.000000,64.000000,128.000000,85.000000,...,119.000000,8.000000,32.000000,0.000000,0.0,0.000000,0.000000,0.0,4.400000,2.700000
max,66771.000000,2017.000000,48.000000,164.000000,162.000000,778.000000,716.000000,152.000000,262.000000,225.000000,...,905.000000,54.000000,221.000000,35.000000,0.0,155.000000,89.000000,0.0,121.400000,48.200000


#### Summary:

For this project, I will be using csv data sets from Baseball Prospectus (containing primarily offensive player stats) and Lahman (contains defensive player stats).  Merging the dataframes required the conversion of the player name convention used by Baseball Prospectus (full first and last name) into the Lahman convention (first 5 letters of last name followed by first two letters of first name).  

The merged dataframe had a number of instances where a particular player would play for more than one team in the course of a season.  For these occurences, I merged the rows for all the teams played in a single season.  

There were also a number of instances where a player will play multiple positions in the course of a single season; for these players, I have assigned them to the position that they have played the most inning outs for the season and aggregated all their defensive stats to that position.

The final data set post data wrangling is a dataframe where each row represents all the statistics of a player for a single sesason.  It contains 18,621 unique player seasons and covers the span of 40 MLB seasons (1977 through 2017).

__General Statistics:__<br>
POS - Position<br>
G - Games Played<br>
GS - Games Started<br>
League - Identifies American or National League<br>

__Offensive Statistics:__<br>
PA - Plate Appearances<br>
AB - At Bats<br>
R - Runs<br>
H - Hits<br>
1B - Singles<br>
2B - Doubles<br>
3B - Triples<br>
HR - Home Runs<br>
TB - Total Bags<br>
BB - Walks<br>
IBB - Intentional Walks<br>
SO - Strike Outs<br>
SF - Sacrifice Flys<br>
SH - Sacrifice Bunt<br>
RBI - Runs Batted In<br>
DP - Double Plays<br>
SB - Stolen Bases<br>
CS - Caught Stealing<br>
AVG - Batting Average<br>
OBP - On Base Percentage<br>
SLG - Slugging<br>
OPS - On Base Plus Slugging<br>
ISO - Isolated Power<br>
oppOPS - Aggregate OPS of pitchers faced<br>
DRC+ - Deserved Runs Created Plus<br>
BRR - Base Running Runs<br>
BWARP - Wins Above Replacement<br>

__Defensive Statistics:__<br>
InnOuts - Defensive Outs Played<br>
Def_PO - Put Outs<br>
Def_A - Assists<br>
Def_E - Errors<br>
Def_DP - Double Play<br>
Def_PB - Passed Ball (Catchers only)<br>
Def_WP - Wild Ptiches (Catchers only)<br>
Def_SB - Stolen Bases (Catchers only)<br>
Def_CS - Caught Stealing (Catchers only)<br>
Def_ZR - Zone Rating<br>
Def_DRAA - Defensive Runs Above Avg<br>
Def_FRAA - Fielding Runs Above Avg<br>